In [1]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import dalmatian
from google.cloud import storage
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

import numpy as np

import sys
import os
sys.path.append('..')  
import model_helper 
from helper import *
from plotly_helper import *

In [2]:
# Function to load data
def load_data(patient_id, wbc_file, treatment_file):
    # Load WBC and treatment data
    wbc_df = pd.read_csv(wbc_file)
    treatment_df = pd.read_csv(treatment_file, sep='\t')
    
    # Get input files from Terra through dalmatian
    workspace = 'broad-firecloud-ibmwatson/TAG_CLL_Clonal_Kinetic_UMI_PrAN'
    wm = dalmatian.WorkspaceManager(workspace)
    participants = wm.get_participants()
    
    # Load additional data for the patient
    cluster_CCF_df = pd.read_csv(participants.loc[patient_id]['cluster_ccfs'], sep='\t')
    
#     tree_df = pd.read_csv(participants.loc[patient_id]['tree_tsv'], sep='\t')
    
    tree_df = pd.read_csv(f"../Cell_Population/{patient_id}_build_tree_posteriors.tsv", sep = '\t')

    abundance_file = f"../Cell_Population/{patient_id}_cell_population_abundances.tsv"
    mcmc_file = f"../Cell_Population/{patient_id}_cell_population_mcmc_trace.tsv"  
    
    abundance_df = pd.read_csv(abundance_file, sep='\t')
    mcmc_df = pd.read_csv(mcmc_file, sep='\t')  
    
    return wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df

# Function to filter data for a specific patient
def filter_patient_data(wbc_df, patient_id):
    wbc_df_patient = wbc_df[wbc_df['Patient'] == patient_id]
    wbc_df_patient.reset_index(drop=True, inplace=True)
    times_sample = [int(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'Time_since_start_tx'].tolist()]
    wbc_all = [float(i) for i in wbc_df_patient['WBC'].tolist()]
    
    
    CLL_count_sample = [float(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'CLL count estm'].tolist()]
    all_times = [int(i) for i in wbc_df_patient['Time_since_start_tx'].to_list()]
    
    return wbc_df_patient, times_sample, CLL_count_sample, wbc_all,all_times





In [3]:
end_treatment_df = pd.read_csv("../CLL8_end_treatment.csv")

In [48]:
# Input patient information
# Input patient information
patient_id = 'CLL14-1161'
wbc_file = '../Copy of CLL14 CLL cell count 07102025 v4_Liang.csv'
treatment_file = '../GCLL_treatment.txt'
UMI_start = 1
UMI_end = 5
treatment_start = 0


In [5]:
# Manually verify the sample list order is correct

In [6]:

sample_list = ['CLL14-1161-Tumor-01-pair', 'SM-ODX25', 'SM-ODX26', 'SM-ODX27', 'SM-ODX28', 'CLL14-1161-Tumor-02-pair'] 

In [7]:
# Output file

In [7]:
outputfile_name = "CLL14_subplot_combined_1161_wbc_updated.html"

Update patient specific information in the below cell 

In [14]:
# Get the treatment duration 

treatment_end = 365

In [15]:
treatment_end

365

In [52]:

wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df = load_data(patient_id, wbc_file,treatment_file )

# Filter data for the specific patient
wbc_df_patient, times_sample, CLL_count_sample,wbc_count, all_times = filter_patient_data(wbc_df, patient_id)

# Plot CLL count over time and save as HTML
CLL_plot_html = plot_CLL_count(patient_id, times_sample, CLL_count_sample, UMI_start, UMI_end, treatment_start, treatment_end, )
    




In [53]:
times_sample

[0, 1035, 1212, 1392, 1575, 1576]

In [54]:
wbc_count

[9.5,
 9.5,
 4.78,
 11.6,
 8.5,
 9.3,
 10.3,
 9.0,
 8.1,
 8.2,
 12.0,
 11.35,
 15.93,
 15.93]

In [22]:
CLL_count_sample


[9.5, 0.492918, 2.449824, 4.319446, 9.389688, nan]

In [23]:
wbc_table_html = plot_metadata_table(wbc_df_patient, patient_id )

In [26]:
tree_df

,n_iter,likelihood,edges
0,145,"[0.3333333333333333, 0.3333333333333333, 0.5, ...","1-2,1-4,2-3,4-5,4-6,None-1"
1,104,"[0.33333333333333337, 0.5, 0.3333333333333332,...","1-2,1-4,2-3,4-5,5-6,None-1"


In [27]:
 plot_tree_html = plot_tree_plotly(tree_df, 1)

In [28]:
 plot_ccf_html = plot_ccf(cluster_CCF_df, times_sample, treatment_df,)

In [29]:
branch_annotations = {
    1: "MAP2K2",
    3:"ASXL2"
    
}

In [30]:
ccf_tree_html = plot_ccf_tree_combined(
    tree_df=tree_df,
    tree_selected=1,
    ccf_df=cluster_CCF_df,
    times_sample=times_sample,
    treatment_df=treatment_df,
    branch_annotations=branch_annotations

)

In [67]:
cluster_list, cluster_abundance = model_helper.get_abundance(abundance_df, mcmc_df, sample_list)
subclone_sample, log_subclone_sample = model_helper.calc_subclone(CLL_count_sample, cluster_abundance, cluster_list)
all_abundance = model_helper.get_all_abundance(cluster_list, mcmc_df, sample_list, times_sample, )

In [68]:
#  Add noise to all abundance

import random 
random.seed(42)
def add_uniform_noise_and_normalize(lst, low=0, high=0.01):
    
    # Add uniform noise
    noisy_lst = [x + random.uniform(low, high) for x in lst]

    
    # Normalize the list
    total_sum = sum(noisy_lst)
    
    normalized_lst = [x / total_sum for x in noisy_lst]

    return normalized_lst


subclone_cluster_iter = {}
# Iterate through each cluster
for cluster, iterations in all_abundance.items():
   
    # Iterate through each iteration in the cluster
    for iteration, value in iterations.items():
        # If the iteration is not in the new dict, add it
        if iteration not in subclone_cluster_iter:
            subclone_cluster_iter[iteration] = {}
        # Add the cluster and its value to this iteration
        subclone_cluster_iter[iteration][cluster] = value

        
        
noise_added_iter = {}

for iteration, abundances in subclone_cluster_iter.items():
    
    transposed_data = {i: list(values) for i, values in enumerate(zip(*abundances.values()), start=1)}
    
    noise_added_iter[iteration] = {}
    for time_point, abundance in transposed_data.items():
        
#         print(time_point, abundance)
        noise_abundance = add_uniform_noise_and_normalize(abundance)
    
#         print(noise_abundance)

        noise_added_iter[iteration][time_point] = add_uniform_noise_and_normalize(abundance)

    
noise_added_all_abundance = {}
for iteration, abundances in noise_added_iter.items():
    original_format = {i + 1: list(values) for i, values in enumerate(zip(*abundances.values()))}
    noise_added_all_abundance[iteration] = original_format
    
    
noise_added_original_all_abundances = {}

for iteration, abundances in noise_added_all_abundance.items():
    
    for cluster, abundance_value in abundances.items():
        if cluster not in noise_added_original_all_abundances:
            noise_added_original_all_abundances[cluster] = {}
        noise_added_original_all_abundances[cluster][iteration] = abundance_value
            

In [69]:
subclone_sample_mcmc_with_uniform_noise, log_subclone_sample_mcmc_with_uniform_noise = model_helper.calc_subclone(CLL_count_sample, noise_added_original_all_abundances, cluster_list, input_type = "mcmc")

In [66]:
times_aft_tx = [x for x in all_times if x > treatment_end]
times_aft_tx.insert(0, treatment_end)

# Set the extrapolate time after treatment
extrapolate_start_idx = 1

In [62]:
times_sample

[0, 1035, 1212, 1392, 1575, 1576]

In [37]:
times_sample[extrapolate_start_idx:]

[1035, 1212, 1392, 1575, 1576]

In [38]:
# Generate Plotly plot
subclone_plot_html = plot_subclones(cluster_list, times_sample, CLL_count_sample, log_subclone_sample, extrapolate_start_idx, times_aft_tx, treatment_df, treatment_end)


In [40]:
linear_model_mcmc_html = plot_linear_model_mcmc(cluster_list,times_sample,  CLL_count_sample, log_subclone_sample_mcmc_with_uniform_noise,extrapolate_start_idx,times_aft_tx,treatment_df, treatment_end)  

In [55]:
# Convert the column "CLL count estm" to float 
wbc_df_patient['CLL count estm'] = wbc_df_patient['CLL count estm'].astype(float) 
times_sliced_aft = [int(i) for i in wbc_df_patient[wbc_df_patient['CLL count estm'] > 0]['Time_since_start_tx'].values  if int(i) > 0]

/var/folders/x6/7zg8fddx4dlgx2lv4sv34_g40000gp/T/ipykernel_23542/981427378.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [56]:
times_aft_tx

[365, 408, 499, 590, 681, 864, 1035, 1212, 1392, 1575, 1576]

In [57]:
# Needs to manually check times_sliced_aft

In [58]:
times_sliced_aft

[408, 499, 590, 681, 1035, 1212, 1392, 1575, 1576]

In [59]:
len(times_sliced_aft)

9

In [70]:
log_subclone_sample_mcmc_with_uniform_noise

{1: {0: [1.0286562871813971,
   -5.539151911154459,
   -3.949624041559247,
   -4.5894630964579965,
   -2.8102891726317676,
   1.401490457059852],
  1: [0.9165556708777106,
   -7.457455610815768,
   -4.103732527669319,
   -3.2564258815825364,
   -5.909299507857427,
   1.41190370717661],
  2: [0.8149743737987203,
   -6.772160333174252,
   -3.9342440669270036,
   -3.3033040797202773,
   -2.402213508474761,
   1.486400033419662],
  3: [0.5084355469548623,
   -6.743840490246638,
   -5.126034349003621,
   -4.46529771237576,
   -2.584277089879552,
   1.5057607046403416],
  4: [0.962541709728197,
   -6.495188537111783,
   -5.556012430431748,
   -3.3754978510094387,
   -5.775222285700239,
   1.448990748899661],
  5: [0.9228949444238747,
   -6.467544383956683,
   -3.826294698069313,
   -3.8977672306862514,
   -3.947954755492379,
   1.4761247631226437],
  6: [0.9645116230926283,
   -5.441683125379087,
   -4.9969482224820165,
   -4.026097377335415,
   -3.238350626135298,
   1.5546601889200447],
  

In [71]:
index_samples_model = slice(1,6)

# index_samples_model is the index of the samples after treatment. For instance, slice(1,7) means sample 2, 3, 4, 5,6, 7

X, y = create_inputs(times_sliced_aft, log_subclone_sample_mcmc_with_uniform_noise,2, index_samples_model, times_sample)



In [72]:
y

[None,
 None,
 None,
 None,
 array([-6.77216033, -1.58262984, -1.26653749, -7.85326809, -6.73497824,
        -5.72859725]),
 array([-3.93424407,  0.09996233,  0.22027104, -4.55107818, -3.37268917,
        -3.34283827]),
 array([-3.30330408,  0.33807884,  0.5996074 , -1.03102937, -0.42979212,
        -2.91253973]),
 array([-2.40221351,  1.13902951,  0.9195478 ,  0.14815592,  0.87868405,
        -2.24913653]),
 array([ 1.48640003, -1.27099992,  1.16313389,  0.88092892,  0.33801574,
         1.44045283])]

In [73]:
wbc_aft_tx = wbc_df_patient[wbc_df_patient.Time_since_start_tx > treatment_end]['CLL count estm'].to_list()
wbc_model = [i for i in wbc_aft_tx if i > 0]

In [74]:
wbc_model

[0.001893,
 0.00914,
 0.00919,
 0.009495,
 0.492918,
 2.449824,
 4.319446,
 9.389688,
 15.93]

In [75]:
len(wbc_model)

9

In [76]:
n_clusters = len(cluster_list)
logsumexp_points = np.log(wbc_model)
model = MultiClusterLinearRegression(n_clusters, X, y)
model.fit(logsumexp_points)

In [77]:
# Generate Plotly plot
subclone_plot_new_model_html = plot_subclones_new_model(cluster_list, times_sample, wbc_model, log_subclone_sample, extrapolate_start_idx, times_aft_tx, times_sliced_aft, treatment_df,treatment_end, model )


In [78]:
plot_mcmc_new_model_html = plot_mcmc_model(cluster_list, index_samples_model, times_aft_tx, times_sample, times_sliced_aft, sample_list, wbc_model, log_subclone_sample_mcmc_with_uniform_noise, treatment_df, treatment_end)


In [79]:
create_html_file([wbc_table_html, plot_ccf_html, ccf_tree_html, CLL_plot_html, subclone_plot_html,linear_model_mcmc_html,subclone_plot_new_model_html, plot_mcmc_new_model_html], output_file=outputfile_name)

HTML file saved as CLL14_subplot_combined_1161_wbc_updated.html
